# Building hybrid model that combines CNN-LSTM.

In [ ]:
#import libraries 
from google.colab import files
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
import re
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import initializers
from keras.layers import Dense, Embedding, LSTM, SimpleRNN, SpatialDropout1D
from keras.layers import Flatten, Conv1D, MaxPooling1D, Dropout, Bidirectional, GRU, SpatialDropout1D
from sklearn.model_selection import train_test_split
from sklearn import metrics 
from sklearn.metrics import f1_score
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from keras.metrics import Precision, Recall
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

In [ ]:
# Upload the CSV file
uploaded = files.upload()

Saving Tweets.csv to Tweets.csv


In [ ]:
data = pd.read_csv("Tweets.csv")

In [ ]:
data = data[['text', 'airline_sentiment']]
data.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [ ]:
#excluding neutral tweets 
data = data[data['airline_sentiment'] != 'neutral']
data.head()

# Tweets Pre-processing 

In [ ]:
#noisy removal 
def clean_train_data(x):
    text = x
    text = text.lower()
    text = re.sub('\[.*?\]', '', text) # remove square brackets
    text = re.sub(r'[^\w\s]','',text) # remove punctuation
    text = re.sub(r'\W', ' ', text) # remove all single characters
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text) # Remove single characters from the start
    text = re.sub('\w*\d\w*', '', text) # remove words containing numbers
    text = re.sub(r'\s+', ' ', text, flags=re.I) # Substituting multiple spaces with single space
    text = re.sub('\n', '', text)
    return text

In [ ]:
data['text'] = data.text.apply(lambda x : clean_train_data(x))
data.head()

In [ ]:
#Remove stopwords 
stop = set (stopwords.words ("english"))
def remove_stopwords (text): 
  text = [word.lower () for word in text.split() if word.lower() not in stop]
  return " ".join(text)

In [ ]:
data["text"] = data["text"].map(remove_stopwords)

In [ ]:
##input/output determination and dataset spiliting 
dataY = pd.get_dummies(data['airline_sentiment']).values
dX_train, dX_test, dY_train, dY_test = train_test_split(data['text'],dataY, test_size = 0.3, random_state = 42)

In [ ]:
#tweet words tokenization
num_words= None
tokenizer = Tokenizer(num_words, split=' ') 
tokenizer.fit_on_texts(data['text'].values) 
X_train = tokenizer.texts_to_sequences(dX_train.values) 
X_train = pad_sequences(X_train)
X_test = tokenizer.texts_to_sequences(dX_test.values) 
X_test = pad_sequences(X_test)

In [ ]:
#to show how each word assigns to value after tokenizer 
word_index = tokenizer.word_index
len (word_index)

12539

In [ ]:
# download pretrained GloVe word embeddings
! wget http://nlp.stanford.edu/data/glove.6B.zip

--2022-08-09 16:50:42--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-08-09 16:50:42--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-08-09 16:50:42--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
! unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [ ]:
embeddings_dictionary = dict()
embedding_dim = 300
glove_file = open('glove.6B.300d.txt')
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()

In [ ]:
#embeddings_dictionary

In [ ]:
#assign each word to its crossponding GloVe word vector 
vocab_len = len(word_index) + 1 
embedding_matrix = np.zeros((vocab_len, embedding_dim))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
#creating f1-score function to perform f1-score calculation
def f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

# Building CNN-LSTM neural network

In [ ]:
embed_dim = 300
lstm_out = 300
max_len=50
model = Sequential()
model.add(Embedding(vocab_len, embed_dim, input_length=None, weights= [embedding_matrix], trainable=False)) 
model.add(SpatialDropout1D(0.5))
model.add(Conv1D(filters=16, kernel_size=2, padding='same', activation='relu', use_bias=True))  #"same" padding option is: output_shape = input_shape / strides
model.add(MaxPooling1D(pool_size=4, strides=1, data_format="channels_last"))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))) 
#model.add(Dense(32, activation='relu'))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy', Precision(), Recall(), f1])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 300)         3762000   
                                                                 
 spatial_dropout1d (SpatialD  (None, None, 300)        0         
 ropout1D)                                                       
                                                                 
 conv1d (Conv1D)             (None, None, 16)          9616      
                                                                 
 max_pooling1d (MaxPooling1D  (None, None, 16)         0         
 )                                                               
                                                                 
 bidirectional (Bidirectiona  (None, 600)              760800    
 l)                                                              
                                                        

In [ ]:
print(X_train.shape,dY_train.shape)
print(X_test.shape,dY_test.shape)

(8078, 24) (8078, 2)
(3463, 26) (3463, 2)


In [ ]:
#model training
batch_size = 32 
model.fit(X_train, dY_train, epochs = 10, batch_size=batch_size, verbose = 2)  

In [ ]:
#model testing 
validation_size = 1500
X_validate = X_test[-validation_size:]
Y_validate = dY_test[-validation_size:]
X_test = X_test[:-validation_size]
dY_test = dY_test[:-validation_size]
score,acc, Precision, Recall, f1 = model.evaluate(X_test, dY_test) # verbose = 2, batch_size = batch_size
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))
print("precision: %.2f" % (Precision))
print("recall: %.2f" % (Recall))
print("f1: %.2f" % (f1))

15/15 [==============================] - 0s 22ms/step - loss: 0.1725 - accuracy: 0.9352 - precision: 0.9352 - recall: 0.9352 - f1: 0.9351
score: 0.17
acc: 0.94
precision: 0.94
recall: 0.94
f1: 0.94


In [ ]:
# Predictions, applying the trained cnn-lstm model to new tweets
predictions = model.predict(X_test)
predictions = np.round(predictions).astype(int)
predictions=np.argmax(predictions, axis=1, out=None)

In [ ]:
df_test = pd.DataFrame(dX_test[:-validation_size]).reindex()
df_test["target"] = predictions

In [ ]:
# Creating submission file 
#submission = pd.DataFrame( data['text'])
df_test.to_csv('Tweets.csv', index=False)
df_test.head(20)

,text,target
11825,usairways charged flight cancelled flightled u...,0
8105,jetblue great flight great view,1
1279,united theyre actually gate agent rude standin...,0
14343,americanair worries called back hrs late fligh...,0
3829,united thank one months ago none weird dont cl...,0
2590,united brothers luggage lost copa airlines fli...,0
11912,americanair great thanks followed,1
11544,usairways work hard making sure things flow sm...,0
3864,united told lostyou guys dont know,0
7940,jetblue bein calling jetblue respond leave num...,0
